## Example Notebook for TreeLearn


Thank you for your interest in our TreeLearn method! With this notebook and google colab, you can try out the pipeline for segmenting a forest point cloud without installing anything on your own computer! However, to actually use our method, we recommend to set up the environment on a gpu-capable device and run the segmentation pipeline there, as described in our repository.

You need to be signed in with your google account. Please also make sure that you are connected to a gpu runtime by by selecting 'runtime' change runtime to e.g. T4 GPU. The following code snippet will show a table with gpu information if you are connnected to a gpu runtime. To run the code snippet, simply click on the left edge. or press (Ctrl + enter) after selecting it.

In [ ]:
!nvidia-smi

The following two code snippets are necessary to set up the environment and download the model weights. Simply run them before continuing. It takes 2 to 3 minutes.

In [2]:
%%capture
# install environment
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install timm==0.6.12
!pip install tensorboard
!pip install tensorboardX
!pip install laspy[lazrs]==2.5.1
!pip install munch==2.5.0
!pip install pandas==2.0.0
!pip install plyfile==0.9
!pip install pyyaml==6.0
!pip install scikit-learn==1.2.2
!pip install six==1.16.0
!pip install tqdm==4.65.0
!pip install open3d-cpu==0.17.0 --default-timeout=100
!pip install jakteristics==0.5.1
!pip install shapely==2.0.1
!pip install geopandas==0.12.2
!pip install alphashape==1.3.1
!pip install spconv-cu114 --default-timeout=100

In [3]:
%%capture
!git clone https://github.com/ecker-lab/TreeLearn.git
%cd TreeLearn
!pip install -e .
%cd ..
!mkdir data
!mkdir checkpoints
!mkdir pipeline
!mkdir pipeline/forests
path = "/content/checkpoints/model_weights.pth"
!wget https://data.goettingen-research-online.de/api/access/datafile/:persistentId?persistentId=doi:10.25625/VPMPID/1JMEQV -O $path

%cd TreeLearn

## Pipeline

We first need to decide which point cloud we want to segment. The following code snippet downloads an example point cloud segment that we did not train on.
If you want to try out another forest point cloud, replace the download with your own. Make sure that the file is in the .npy, .npz, .las, .laz or .txt file format and the total size of the forest plot should be around 1600 m^2 at maximum. Please note that with a forest point cloud of this size the segmentation took in our runs around 15 minutes in google colab due to limited computation resources.

In [ ]:
forest_name = "plot_7_cut.laz"
path = "/content/pipeline/forests/" + forest_name
!wget https://data.goettingen-research-online.de/api/access/datafile/:persistentId?persistentId=doi:10.25625/VPMPID/0WDXL6 -O $path


To run the TreeLearn pipeline interactively in google colab, we import the function run_treelearn_pipeline. This function takes as argument the config dict. 
We import the pipeline.yaml as the config dict and print it. We adjust some entries in the config dict to fit to the setting in google colab and speed up the pipeline. We also initialize the logger so that the progress in the pipeline is printed.

In [ ]:
import sys
sys.path.append("/content/TreeLearn/tools/pipeline")
from pipeline import run_treelearn_pipeline
import argparse, pprint
from tree_learn.util import get_config

config_path = "/content/TreeLearn/configs/pipeline/pipeline.yaml"
config = get_config(config_path)

# adjust config
config.forest_path = "/content/pipeline/forests/" + forest_name
config.dataset_test.data_root = "/content/pipeline/tiles"
config.tile_generation = True
config.pretrain = "/content/checkpoints/model_weights.pth"
config.sample_generation.stride = 0.9 # small overlap of tiles
config.shape_cfg.outer_remove = False # default value = 13.5
config.save_cfg.save_treewise = False
config.save_cfg.return_type = "voxelized_and_filtered"
print(pprint.pformat(config.toDict(), indent=2))

import logging
logger = logging.getLogger("TreeLearn")
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
logging.basicConfig()
ch = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel(logging.INFO)

After having set all the correct settings in the config file, it remains to run the pipeline. Please keep in mind that fully running it for the example point cloud takes around 12 minutes.



In [ ]:
# run pipeline
run_treelearn_pipeline(config)
# runtime ~ 12 min

If everything has run as expected, the segmented point cloud is now saved in the .laz format with labels in the directory /content/pipeline/results. It is also saved in the .npz format. You can easily download it by right-clicking and selecting download. **Please note that, to speed up the pipeline for demonstration purposes, we do not remove a buffer of 13.5 meters at the edge of the input point cloud. This is usually recommended since points at the edge do not have the necessary context for making accurate predictions. The predictions at the edge of the example point cloud do not represent the capabilities of TreeLearn.**

In [1]:
import laspy
import torch

In [2]:
print(torch.__version__)

1.9.1+cu111


In [4]:
las_file = laspy.read('/home/pengzhen/code/pointcloud_dataset_set/dataset/TreelearnDataset/pipeline/L1W/forest/L1W.laz')

LaszipError: reading point 0 of 85289646 total points